In [1]:
import os 

In [2]:
%pwd

'/Users/ravina/Desktop/CustomerChurnPrediction/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/ravina/Desktop/CustomerChurnPrediction'

In [5]:
import mlflow

mlflow.set_tracking_uri("https://dagshub.com/ravina029/CustomerChurnPrediction.mlflow")


In [6]:
os.environ["MLFLOW_TRACKING_URI"]='https://dagshub.com/ravina029/CustomerChurnPrediction.mlflow'
os.environ["MLFLOW_TRACKING_USERNAME"]='ravina029'
os.environ["MLFLOW_TRACKING_PASSWORD"]='221d4e3a527ff8b9aef06e059d7efc4e89963e11'


In [7]:
#Entity of model evaluation
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name:Path
    target_column: str
    mlflow_uri: str
    

In [8]:
from CustomerChurnPrediction.constants import *
from CustomerChurnPrediction.utils.common import read_yaml,create_directories, save_json


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.RandomForestClassifier
        #params=self.params.XGBClassifier
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/ravina029/CustomerChurnPrediction.mlflow"
            )

        return model_evaluation_config

In [10]:
#components

import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score,f1_score
from urllib.parse import urlparse
from sklearn.utils.validation import check_X_y
#from sklearn.utils.multiclass import type_of_target
import mlflow
import mlflow.sklearn
import numpy as np
import joblib



In [11]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config
        

    def eval_metrics(self,actual,pred):
            accuracy = accuracy_score(actual, pred)
            precision = precision_score(actual, pred)
            recall = recall_score(actual, pred)
            roc_auc = roc_auc_score(actual, pred)
            f_Score=f1_score(actual,pred)
            return accuracy,precision,recall,roc_auc,f_Score
        

    def log_into_mlflow(self):
            test_data=pd.read_csv(self.config.test_data_path)
            model=joblib.load(self.config.model_path)

            test_x=test_data.drop([self.config.target_column],axis=1)
            test_y=test_data[[self.config.target_column]]

            print("shape of test dataset",test_x.shape,test_y.shape)
            print("columns of test_x are",test_x.columns)
            
            
            


            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_score=urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                predicted_qualities=model.predict(test_x)
                #y_prob = model.predict_proba(test_x)[:, 1]
                #roc_auc = roc_auc_score(test_y, y_prob) #as our data is highly imbalanced, therefore using this method using y_prob.
                (accuracy,precision,recall,roc_auc,f_Score)=self.eval_metrics(test_y,predicted_qualities)
                
                #saving metrics as local
                scores = {"accuracy": accuracy, "precision_score": precision, "Recall_score": recall, "Roc_Auc_score": roc_auc,"f1_score":f_Score}
                print(scores)
                save_json(path=Path(self.config.metric_file_name), data=scores)
                mlflow.log_params(self.config.all_params)

                
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision_score", precision)
                mlflow.log_metric("Recall_score", recall)
                mlflow.log_metric("Roc_Auc_score",roc_auc)
                

                #Model registry doesn't work with file store
                if tracking_url_type_score!='file':
                    #Regitster the model
                    #There are other ways to use the Model registry, which depends on the use case,
                    #please refer to the doc for more information:params=self.params.XGBClassifier
                    #https://mlflow.org/docs/latest/models.html
                    mlflow.sklearn.log_model(model,"model", registered_model_name='RandomForestClassifier')
                    #mlflow.sklearn.log_model(model,"model", registered_model_name='XGBClassifier')
                    
                else:
                    mlflow.sklearn.log_model(model,'model') 



In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e


[2024-04-30 14:27:42,572:INFO:yaml file:config/config.yaml loaded successfully]
[2024-04-30 14:27:42,574:INFO:yaml file:params.yaml loaded successfully]
[2024-04-30 14:27:42,576:INFO:yaml file:Schema.yaml loaded successfully]
[2024-04-30 14:27:42,576:INFO:created directory at:artifacts]
[2024-04-30 14:27:42,577:INFO:created directory at:artifacts/model_evaluation]
shape of test dataset (3000, 11) (3000, 1)
columns of test_x are Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary',
       'Geography', 'Gender', 'NumOfProducts', 'IsActiveMember',
       'sufficient_balance', 'is_CreditScore_low'],
      dtype='object')
{'accuracy': 0.8246666666666667, 'precision_score': 0.5548387096774193, 'Recall_score': 0.7037643207855974, 'Roc_Auc_score': 0.7796762164832132, 'f1_score': 0.6204906204906205}
[2024-04-30 14:27:44,341:INFO:json file saved at:artifacts/model_evaluation/metrics.json]


/Users/ravina/Desktop/CustomerChurnPrediction/churnvenv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/ravina/Desktop/CustomerChurnPrediction/churnvenv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/04/30 14:28:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestClassifier, version 15
Created version '15' of model 'RandomForestClassifier'.
